In [1]:
%load_ext autotime


time: 202 µs


In [2]:
import numpy as np
import _pickle as pickle
import pandas as pd

time: 1.05 s


In [3]:
print("==========LOADING DATA==========")
age_seq = pickle.load(open("../data/new_age_seq","rb"))
sex_seq = pickle.load(open("../data/new_sex_seq","rb"))

util_seq = pickle.load(open("../data/new_util_seq","rb"))
code_seq = pickle.load(open("../data/new_code_seq","rb"))
date_seq = pickle.load(open("../data/new_date_seq","rb"))
label_seq = pickle.load(open("../data/new_label_seq","rb"))

==========LOADING DATA==========
time: 41.8 ms


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

time: 5.04 s


In [5]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, seqs, vocab_sizes, list_IDs, max_visit, max_code, batch_size=100, shuffle=True):
        self.seqs = seqs
        self.code_vocab = vocab_sizes[0]
        self.cat_vocab = vocab_sizes[1]
        self.list_IDs = list_IDs
        self.max_visit = max_visit
        self.max_code = max_code
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch' 
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' 
        demo_feature, code_feature, util_feature, date_feature, cls_feature = self.seqs
        batch_demo, batch_code, batch_util, batch_date, batch_cls = [], [], [], [], []
        for i, ID in enumerate(list_IDs_temp):
            batch_demo.append(demo_feature[ID])
            batch_code.append(code_feature[ID])
            batch_util.append(util_feature[ID])
            batch_date.append(date_feature[ID])
            batch_cls.append(cls_feature[ID])
        
        batch_demo_feature = np.array(batch_demo)
        batch_code_feature = self.code_padding(batch_code)
        batch_util_feature = self.date_padding(batch_util)
        batch_date_feature = self.date_padding(batch_date)
        batch_cls = np.array(batch_cls)
        
        dic = (
            {
                'demo_feature': batch_demo_feature,
                'code_feature': batch_code_feature,
                'util_feature': batch_util_feature,
                'date_feature': batch_date_feature,
            },
            {
                'cls_label': batch_cls
            })
        return dic
    
    def date_padding(self, seq):
        seq = [x[:-1] for x in seq]
        
        pad_seq = np.zeros((len(seq), self.max_visit))
        for i, p in enumerate(seq):
            pad_seq[i][:len(p)] = p[:self.max_visit]
        return pad_seq
    
    def code_padding(self, seq):
        seq = [x[:-1] for x in seq]
        
        X = np.zeros((len(seq), self.max_visit, self.max_code))
        for i, p in enumerate(seq):
            if len(p) > self.max_visit: 
                p = p[:self.max_visit]
            for j, claim in enumerate(p):
                claim = claim[:self.max_code]
                X[i][j][:len(claim)] = claim
        return X


time: 10 ms


In [6]:
def create_code_mask(code_seq):
    code_mask = tf.cast(tf.math.not_equal(code_seq, 0), tf.float32)
    return code_mask[:,:,:,tf.newaxis]

def create_visit_mask(seq):
    visit_mask = tf.cast(tf.math.not_equal(seq, 0), tf.float32)
    return visit_mask[:,:]

def scaled_dot_product_attention(Q, K, V, Q_masks, K_masks):
    d_k = K.get_shape().as_list()[-1] # d_model/h

    outputs = tf.matmul(Q, tf.transpose(K, [0, 2, 1]))  # (h*N, T_q, T_k)
    outputs /= d_k ** 0.5

    padding_num = -1e+7
    K_masks = tf.expand_dims(K_masks, 1) # (h*N, 1, T_k)
    K_masks = tf.tile(K_masks, [1, tf.shape(Q)[1], 1]) # (h*N, T_q, T_k)
    paddings = tf.ones_like(outputs) * padding_num
    outputs = tf.where(tf.equal(K_masks, 0), paddings, outputs)  # (h*N, T_q, T_k)

    outputs = tf.nn.softmax(outputs)
    attention = outputs
    Q_masks = tf.expand_dims(Q_masks, -1) # (h*N, T_q, 1)
    Q_masks = tf.tile(Q_masks, [1, 1, tf.shape(K)[1]]) # (h*N, T_q, T_k)
    outputs = outputs * tf.cast(Q_masks, dtype=tf.float32)

    return tf.matmul(outputs, V), attention # [h*N, T_q, d_model/h]

class multihead_attention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, name="multihead_attention"):
        super(multihead_attention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.query_dense = layers.Dense(units=d_model, use_bias=False)
        self.key_dense = layers.Dense(units=d_model, use_bias=False)
        self.value_dense = layers.Dense(units=d_model, use_bias=False)
        self.add =layers.Add()
        self.norm = layers.LayerNormalization()
    
    def call(self, queries, keys, values, query_masks, key_masks):
        Q = self.query_dense(queries)
        K = self.key_dense(keys)
        V = self.value_dense(values)

        # Split and concat
        Q_ = tf.concat(tf.split(Q, self.num_heads, axis=2), axis=0) # (h*N, T_q, d_model/h)
        K_ = tf.concat(tf.split(K, self.num_heads, axis=2), axis=0) # (h*N, T_k, d_model/h)
        V_ = tf.concat(tf.split(V, self.num_heads, axis=2), axis=0) # (h*N, T_v, d_model/h)
        query_masks = tf.tile(query_masks, [self.num_heads, 1]) # (h*N, T_q)
        key_masks = tf.tile(key_masks, [self.num_heads, 1]) # (h*N, T_k)

        # Attention
        outputs, attention = scaled_dot_product_attention(Q_, K_, V_, query_masks, key_masks) # (h*N, T_q, d_model/h)

        # Restore shape
        outputs = tf.concat(tf.split(outputs, self.num_heads, axis=0), axis=2) # (N, T_q, d_model)

        # Residual connection
        outputs = self.add([queries, outputs])
        outputs = self.norm(outputs)
        
        return outputs, attention

class ffn(tf.keras.layers.Layer):
    def __init__(self, d_model, ffn_dim, name="ffn"):
        super(ffn, self).__init__(name=name)
        self.ffn_dim = ffn_dim
        self.dense1 = layers.Dense(units=ffn_dim, activation=tf.nn.relu, use_bias=False)
        self.dense2 = layers.Dense(units=d_model, use_bias=False)
        self.add =layers.Add()
        self.norm = layers.LayerNormalization()
    
    def call(self, inputs):
        outputs = self.dense1(inputs)
        outputs = self.dense2(outputs)
        outputs = self.add([inputs, outputs])
        outputs = self.norm(outputs)
        return outputs

def cat_recall(y_true, y_pred):
    mask_value = tf.cast(tf.not_equal(tf.reduce_sum(y_true,axis=-1), 0), tf.float32)
    true_positives = tf.cast(tf.reduce_sum(tf.multiply(tf.round(y_pred), y_true), axis=-1), tf.float32)
    possible_positives = tf.cast(tf.reduce_sum(y_true, axis=-1), tf.float32)
    values = true_positives / (possible_positives + 1e-7)
    return tf.reduce_sum(values)/tf.reduce_sum(mask_value)

def cat_loss_fun(y_true, y_pred):
    loss = tf.cast(tf.keras.losses.BinaryCrossentropy(reduction='none')(y_true, y_pred), tf.float32)
    mask = tf.cast(tf.not_equal(tf.reduce_sum(y_true,axis=-1), 0), tf.float32)
    loss = tf.multiply(loss, mask)
    # return tf.reduce_sum(loss)/tf.reduce_sum(mask)
    return loss

def model(
    max_visit,
    max_code,
    max_demo,
    
    demo_vocab,
    code_vocab,
    date_vocab,
    util_vocab,
    cat_vocab,

    patient_dim,
    vocab_dim=100,
    model_dim=100,
    ffn_dim=100,
    num_heads=2,
    num_translayer=1,
    
    model_name="TransF"):
    
    demo = layers.Input(shape=(max_demo, ), name="demo_feature")  # max_demo = 2, age&sex
    code_seq = layers.Input(shape=(max_visit, max_code), name="code_feature") 
    util_seq = layers.Input(shape=(max_visit), name="util_feature")
    date_seq = layers.Input(shape=(max_visit), name="date_feature")

    inputs = [demo, code_seq, util_seq, date_seq]
    
    # demo embedding
    demo_emb = layers.Embedding(input_dim=demo_vocab, output_dim=vocab_dim, mask_zero=True, name='demo_embedding')(demo)
    demo_emb = layers.Lambda(lambda x: tf.keras.backend.sum(x, axis=1))(demo_emb)     

    # code sequence
    code_mask = layers.Lambda(create_code_mask)(code_seq)
    code_emb = layers.Embedding(input_dim=code_vocab, 
                                output_dim=vocab_dim, 
                                name='code_embed')(code_seq)
    code_emb = layers.Multiply()([code_emb, code_mask])
    code_emb = tf.reduce_sum(code_emb, axis=2)     

    
    # visit mask
    visit_mask = layers.Lambda(create_visit_mask)(date_seq)
    
    # util sequence   
    util_emb = layers.Embedding(input_dim=util_vocab, output_dim=vocab_dim, mask_zero=True, name='util_embedding')(util_seq)
    util_emb = layers.Multiply()([util_emb, visit_mask[:,:,tf.newaxis]])
    

    return tf.keras.Model(inputs=inputs, outputs=outputs, name=model_name)



time: 18.3 ms


In [7]:
def process_code(seq, vocab2int):
    unseen = []
    new_seq = []
    for p in seq:
        new_p = []
        for v in p:
            new_v = []
            for c in v:
                if c not in vocab2int: 
                    unseen.append(c)
                    continue
                    # vocab2int[c] = len(vocab2int)
                new_v.append(vocab2int[c])
            new_p.append(new_v)
        new_seq.append(new_p)
        
    print("UNSEEN VOCAB:",len(set(unseen)), len(unseen))
    return new_seq

def process_util(seq, util2int):
    new_seq = []
    vocab2int = {"PAD":0,"IP":1,"RX":2,"OP":3}
    for p in seq:
        new_p = []
        for v in p:
            if "IP" in v:
                new_v=1
            elif "RX" in v:
                new_v=2
            else:
                new_v=3
            new_p.append(new_v)
        new_seq.append(new_p)
    return new_seq
    
def process_demo(age_seq, sex_seq, vocab2int):
    new_seq = []
    for age, sex in zip(age_seq, sex_seq):
        p = []
        assert age in vocab2int
        assert sex in vocab2int
        
        p.append(vocab2int[age])
        p.append(vocab2int[sex])
        new_seq.append(p)
    return np.array(new_seq)

def get_cat(seq,code2cat):
    new_seq = []
    for p in seq:
        new_p = []
        for v in p:
            new_v = []
            for c in v:
                new_c = code2cat[c]
                new_v.append(new_c)
            new_p.append(new_v)
        new_seq.append(new_p)
    return new_seq

time: 27 ms


In [8]:
print("------LOADING DIC------")
path = "/Users/xxz005/Desktop/RAW_DATA/code2cat/"

diag2cat = pickle.load(open(path+"diag2cat","rb"))
proc2cat = pickle.load(open(path+"proc2cat","rb"))
drug2cat = pickle.load(open(path+"drug2cat","rb"))

code2cat = {**diag2cat, **proc2cat, **drug2cat}

------LOADING DIC------
time: 89.1 ms


In [9]:
code2int, util2int, demo2int, cat2int  = pickle.load(open("../../pretraining/model/vocabs/vocabs","rb"))

time: 46.1 ms


In [10]:
code_feature = process_code(code_seq, code2int)
util_feature = process_util(util_seq, util2int)
demo_feature = process_demo(age_seq, sex_seq, demo2int)
date_feature = date_seq

cls_feature = np.array(label_seq).reshape((-1,1))

UNSEEN VOCAB: 0 0
time: 11.3 ms


In [11]:
MAX_VISIT=30
MAX_CODE=10
MAX_DEMO=2
PATIENT_DIM=100

BATCH_SIZE = 500
TRAIN_RATIO = 0.7
DATA_SIZE = len(age_seq)
EPOCHS = 20

params = {
    'seqs':[demo_feature, code_feature, util_feature, date_feature, cls_feature],
    'vocab_sizes': [len(code2int), len(cat2int)],
    'batch_size':100,
    'max_visit':MAX_VISIT, 
    'max_code':MAX_CODE,
}

time: 2.21 ms


In [12]:
from sklearn.model_selection import train_test_split
train_IDs, valid_IDs = train_test_split(range(DATA_SIZE), train_size=TRAIN_RATIO, random_state=42)
train_generator = DataGenerator(list_IDs=train_IDs, shuffle=True, **params)
valid_generator = DataGenerator(list_IDs=valid_IDs, shuffle=False, **params)

time: 1.69 s


# Pretrain

In [13]:
# model_path = "../../pretraining/model/saveModel"

# model = tf.keras.models.load_model(model_path)

time: 417 µs


In [15]:
model_path = "/Users/xxz005/Desktop/saveModel"

model = tf.keras.models.load_model(model_path)

time: 1.67 s


In [16]:
model_losses = {
    "cls_label":tf.keras.losses.BinaryCrossentropy(),
}

model_metrics = {
    "cls_label": tf.keras.metrics.AUC(),
}

time: 7.8 ms


In [17]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss=model_losses, metrics=model_metrics)
print(model.summary())

Model: "TransF"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
demo_feature (InputLayer)       [(None, 2)]          0                                            
__________________________________________________________________________________________________
demo_embedding (Embedding)      (None, 2, 100)       2400        demo_feature[0][0]               
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 100)          0           demo_embedding[0][0]             
__________________________________________________________________________________________________
code_feature (InputLayer)       [(None, 30, 10)]     0                                            
_____________________________________________________________________________________________

In [18]:
model.evaluate(valid_generator)

2/2 [==============================] - 1s 25ms/step - loss: 5.3384 - cls_label_loss: 5.3384 - cls_label_auc: 0.5191


[5.3545451164245605, 5.3545451164245605, 0.5202370285987854]

time: 1.48 s


In [19]:
finetune_history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=valid_generator,
    verbose=2,
)

Epoch 1/20
4/4 - 2s - loss: 2.8303 - cls_label_loss: 2.8303 - cls_label_auc: 0.5014 - val_loss: 0.8291 - val_cls_label_loss: 0.8291 - val_cls_label_auc: 0.5882
Epoch 2/20
4/4 - 0s - loss: 1.3828 - cls_label_loss: 1.3828 - cls_label_auc: 0.4872 - val_loss: 1.3247 - val_cls_label_loss: 1.3247 - val_cls_label_auc: 0.5479
Epoch 3/20
4/4 - 0s - loss: 1.4830 - cls_label_loss: 1.4830 - cls_label_auc: 0.5295 - val_loss: 0.9591 - val_cls_label_loss: 0.9591 - val_cls_label_auc: 0.5396
Epoch 4/20
4/4 - 0s - loss: 0.9104 - cls_label_loss: 0.9104 - cls_label_auc: 0.5244 - val_loss: 0.8159 - val_cls_label_loss: 0.8159 - val_cls_label_auc: 0.5259
Epoch 5/20
4/4 - 0s - loss: 0.8908 - cls_label_loss: 0.8908 - cls_label_auc: 0.5669 - val_loss: 1.0094 - val_cls_label_loss: 1.0094 - val_cls_label_auc: 0.5249
Epoch 6/20
4/4 - 0s - loss: 0.7480 - cls_label_loss: 0.7480 - cls_label_auc: 0.6431 - val_loss: 0.6296 - val_cls_label_loss: 0.6296 - val_cls_label_auc: 0.5945
Epoch 7/20
4/4 - 0s - loss: 0.6394 - cls

In [20]:
outs = model.predict(valid_generator)

time: 442 ms


In [21]:
outs[0].shape, outs[1].shape, outs[2].shape

((147, 3357), (147, 30, 3357), (147, 1))

time: 3.12 ms


# Cold Start

In [144]:
m = model(
    patient_dim=PATIENT_DIM,
    max_visit=MAX_VISIT,
    max_code=MAX_CODE,
    max_demo=MAX_DEMO,
    code_vocab=len(code2int),
    demo_vocab=len(demo2int),
    util_vocab=4,
    date_vocab=365,
    cat_vocab=len(cat2int),
)

time: 512 ms


In [146]:
m.compile(optimizer="RMSprop", loss=tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.Recall(top_k=5), 
                                                                                       tf.keras.metrics.Recall(top_k=10), 
                                                                                       tf.keras.metrics.Recall(top_k=30)])
print(m.summary())


Model: "TransF"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
demo_feature (InputLayer)       [(None, 2)]          0                                            
__________________________________________________________________________________________________
demo_embedding (Embedding)      (None, 2, 100)       2400        demo_feature[0][0]               
__________________________________________________________________________________________________
code_feature (InputLayer)       [(None, 30, 10)]     0                                            
__________________________________________________________________________________________________
date_feature (InputLayer)       [(None, 30)]         0                                            
_____________________________________________________________________________________________

In [147]:
# opt = tf.keras.optimizers.Adam(learning_rate=0.01)
# m.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.Recall(top_k=5), 
#                                                                                        tf.keras.metrics.Recall(top_k=10), 
#                                                                                        tf.keras.metrics.Recall(top_k=30)])
# print(m.summary())

time: 325 µs


In [148]:
cold_start_his = m.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=valid_generator,
    verbose=2,
)


Epoch 1/20
70/70 - 11s - loss: 0.0524 - recall_41: 0.1668 - recall_42: 0.2301 - recall_43: 0.3743 - val_loss: 0.0047 - val_recall_41: 0.1927 - val_recall_42: 0.2697 - val_recall_43: 0.4610
Epoch 2/20
70/70 - 9s - loss: 0.0045 - recall_41: 0.1983 - recall_42: 0.2773 - recall_43: 0.4633 - val_loss: 0.0045 - val_recall_41: 0.1966 - val_recall_42: 0.2817 - val_recall_43: 0.4795
Epoch 3/20
70/70 - 9s - loss: 0.0044 - recall_41: 0.2138 - recall_42: 0.3049 - recall_43: 0.4900 - val_loss: 0.0044 - val_recall_41: 0.2262 - val_recall_42: 0.3118 - val_recall_43: 0.5048
Epoch 4/20
70/70 - 9s - loss: 0.0043 - recall_41: 0.2468 - recall_42: 0.3443 - recall_43: 0.5371 - val_loss: 0.0043 - val_recall_41: 0.2426 - val_recall_42: 0.3308 - val_recall_43: 0.5269
Epoch 5/20
70/70 - 9s - loss: 0.0041 - recall_41: 0.2795 - recall_42: 0.3847 - recall_43: 0.5798 - val_loss: 0.0043 - val_recall_41: 0.2471 - val_recall_42: 0.3388 - val_recall_43: 0.5346
Epoch 6/20
70/70 - 9s - loss: 0.0039 - recall_41: 0.3166 - 